In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null

E: Package 'python-software-properties' has no installation candidate


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/'My Drive'/'CIS 520 Project'

/content/drive/My Drive/CIS 520 Project


This notebook cleans the reviews from restaurant_reviews.csv file and saves the reviews as bag of words along with the corresponding rating. The star and review column name remain the same i.e. "stars_x" and "text". 

1. "stars_x" column is of type int and only contains values from 1-5.
2. "text" column contains a list of words in the review.

In [0]:
#imports
import os
import json
import pandas as pd
from glob import glob
import nltk
import string
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
# from vocabulary import Vocabulary
from nltk.stem import WordNetLemmatizer
#from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
import configparser

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
#reading 'stars_x' and 'text' columns
pd.options.display.max_colwidth = 10000
df = pd.read_csv('restaurant_reviews.csv', usecols = ['stars_x', 'text'], engine='python')

In [0]:
#Checking unique stars_x values
print(df['stars_x'].unique())

In [0]:
df_text = df[['text']]
df_text.isnull()

In [0]:
df.loc[df['stars_x'].isnull() == True]

Setting data types of "stars_x" column as int and "text" column as string.

In [0]:
#removing null value, setting type for columns
df = df.dropna()
df = df.astype({'text':str, 'stars_x':'int32'})

In [0]:
df.info()

Following tasks are performed on the reviews - 
1. spaces stripped
2. \n, \r removed
3. word tokenization
4. conversion of words to lower case
5. removal of stopwords
6. removal of punctuations
7. lemmatization

In [0]:
it = 0
def clean_text(col):
  #run check
  global it
  if (it%100000==0):
    print(it)
  it+=1
  
  lemmatizer = WordNetLemmatizer()
  stopWords = set(stopwords.words('english'))
  #stirp spaces
  col = col.strip()
  #remove /n, /r
  col = col.replace('\r', ' ')
  col = col.replace('\n', ' ')
  words = word_tokenize(col)
  bow = []  
  for word in words:
      word = word.lower()
      if word not in stopWords and word not in string.punctuation:
          new_word = lemmatizer.lemmatize(word)
          bow.append(new_word)
  return bow

In [0]:
def bow_text(df):
  df['text'] = df['text'].apply(clean_text)
  return df

Bag of word reviews are stored in reviews_bow.csv. To read this file using pandas use pandas.read_pickle().

In [0]:
dfnew = bow_text(df)
dfnew.to_pickle('reviews_bow.csv')